In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import datetime

In [34]:
df = pd.read_csv(r'/Users/Angela/Desktop/thursday.csv')
dfOldData = pd.read_csv(r'/Users/Angela/Desktop/output.csv')

In [3]:
df.rt.unique()

array(['BB', 'NW', 'WX', 'MX', 'CN', 'CS', 'WS', 'NE', 'NX', 'DD', 'CSX'],
      dtype=object)

In [4]:
df.stpnm.unique()

array(['Baits II Inbound', 'Bursley Hall Inbound',
       'Hayward/Hubbard  Hubbard Outbound', 'Glen/Catherine Outbound',
       'Fuller Rd at Mitchell Field  Lot M-75',
       'Central Campus Transit Center: Ruthven Museum',
       'Pierpont Commons  Murfin Outbound', 'FXB Inbound',
       'Cooley Lab Inbound', 'Northwood I Outbound',
       'NCAC, Hubbard Outbound',
       'Fuller Rd at Lot NC-78  Mitchell Field (1)', 'Northwood V (3)',
       'Power Center', 'Glen/Catherine Inbound',
       'Pierpont Commons  Bonisteel Inbound', 'Cardiovascular Center',
       'NCAC  Hubbard Inbound', 'Hayward/Hubbard  Hubbard Inbound',
       'Plymouth Rd Crosswalk', 'Northwood Community Center',
       'Northwood III Outbound', 'Northwood II Outbound',
       'Stockwell Hall Outbound', 'Rackham Bldg',
       'Central Campus Transit Center: Chemistry', 'Couzens Hall',
       'Cancer Center Inbound', 'Mott Hospital', 'Markley Hall',
       'Taubman Outbound', 'Cancer Center Outbound', 'Taubman Inbou

In [35]:
def formatting(indata) :
    indata.tmstmp = pd.to_datetime(indata.tmstmp)
    stoptimes = indata.groupby([indata.tmstmp.dt.date, "tatripid","rtdir","stpnm","rt"]).agg("last")
# stoptimes = df.groupby([df.tmstmp.dt.date,'tatripid']).agg("last")
    stoptimes.index.names = ["date", "tatripid","rtdir","stpnm","rt"]
    stoptimes = stoptimes.reset_index(level=[2,3,4])
#     stoptimes['rtdir'] = stoptimes.index.get_level_values('rtdir')
#     stoptimes["stpnm"] = stoptimes.index.get_level_values("stpnm")
#     stoptimes["rt"] = stoptimes.index.get_level_values("rt")
#     stoptimes.tmstmp.reset_index(level=1)
    return stoptimes
# waittimes = stoptimes.tmstmp.reset_index(level=1).groupby(["date"]).agg(lambda x: x.sort_values().diff())
# waittimes

In [6]:
# def formatting(indata, stops, direction) :
#     indata.tmstmp = pd.to_datetime(indata.tmstmp)
#     stoptimes = indata[indata.rtdir.eq(direction) & indata.stpnm.isin(stops)]
#     stoptimes = stoptimes.groupby([stoptimes.tmstmp.dt.date, "tatripid"]).agg("last")
#     stoptimes.index.names = ["date", "tatripid"]
#     return stoptimes

In [40]:
# this is where new files are created
newRoutes = formatting(df)
oldRoutes = formatting(dfOldData)
# newRoutes.to_csv(r'/Users/Angela/Desktop/newRoutes.csv')
# oldRoutes.to_csv(r'/Users/Angela/Desktop/oldRoutes.csv')

In [37]:
def WaittimesCalc(indata, stops, direction) :
    intimes = indata[indata.rtdir.eq(direction) & indata.stpnm.isin(stops)]
    # !! limiting line below
    # idk how to get this work without losing all direction, stop, and bus route info
    waittimes = intimes.tmstmp.reset_index(level=1).groupby("date").agg(lambda x: x.sort_values().diff())
    # !! limiting line above
    waittimes = waittimes.reset_index()
    waittimes = pd.concat([waittimes.drop(['tmstmp'], axis=1), waittimes['tmstmp'].apply(pd.Series)], axis=1)
    waittimes = waittimes.melt(id_vars=["date"])
    waittimes = waittimes[["date", "value"]]
    waittimes["value"] = waittimes.value.dt.total_seconds()/60
    waittimes["zscores"] = (waittimes["value"] - waittimes["value"].mean())/waittimes["value"].std()
    waittimes = waittimes[waittimes.zscores < 3]
    return waittimes

In [38]:
def avgWaitCalc(indata, stops, direction) :
    waittimes = WaittimesCalc(indata, stops, direction)
    return waittimes.value.mean()

In [10]:
def avgWaitCalcRt(indata, stops, direction, routes) :
    rtData = indata[indata.rt.isin(routes)]
    return avgWaitCalc(rtData, stops, direction)

In [11]:
def graphWaittimes(indata, stops, direction) :
    waittimes = WaittimesCalc(indata, stops, direction)
    return sns.displot(waittimes.value)

In [39]:
avgWaitCalc(newRoutes, ["Central Campus Transit Center: Chemistry", "Central Campus Transit Center: Ruthven Museum"] , "NORTHBOUND")

2.6802911534154537

In [41]:
avgWaitCalcRt(newRoutes, ["Central Campus Transit Center: Chemistry", "Central Campus Transit Center: Ruthven Museum"] , "NORTHBOUND", ["BB"])

6.29842180774749

In [42]:
avgWaitCalcRt(newRoutes, ["Central Campus Transit Center: Chemistry", "Central Campus Transit Center: Ruthven Museum"] , "NORTHBOUND", ["DD"])

10.755905511811024

In [43]:
avgWaitCalcRt(newRoutes, ['Crisler Center/Lot SC-7'], 'NORTHBOUND', ['CN'])

10.587223587223587

In [45]:
# this isn't working
avgWaitCalc(oldRoutes, ["Central Campus Transit Center: Ruthven Museum"] , "NORTHBOUND")

6.207868852459017